### Welcome to Week 6 Day 3!

Let's experiment with a bunch more MCP Servers

In [9]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
from datetime import datetime
load_dotenv(override=True)

True

### The first type of MCP Server: runs locally, everything local

Here's a really interesting one: a knowledge-graph based memory.

It's a persistent memory store of entities, observations about them, and relationships between them.

https://github.com/modelcontextprotocol/servers/tree/main/src/memory


In [ ]:
#we always start by specifying the parameters of an mcp
params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:./memory/shreya.db"}}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='create_entities', description='Create new entities with observations', inputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, annotations=None, title='Create new entities with observations'),
 Tool(name='search_nodes', description='Search for entities and their relations using text search with relevance ranking', inputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'query': {'type': 'string'}, 'limit': {'type': 'number'}}, 'required': ['query']}, annotations=None, title='Search for entities and their relations using text search with relevance ranking'),
 Tool(name='read_graph', description='Get recent entit

In [3]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."
request = "My name's Shreya. I'm an LLM engineer. I'm learning about AI Agents, including the incredible MCP protocol. \
MCP is a protocol for connecting agents with tools, resources and prompt templates, and makes it easy to integrate AI agents with capabilities."
model = "gpt-4.1-mini"

In [ ]:
#We use this context manager with MCP Server Studio, which we know is going to create the MCP client.
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    #then we create an agent named "agent" with the model, instructions and mcp server
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request) #then we call Runner.run with the agent and requet
    display(Markdown(result.final_output))

    #a file "shreya.db" has been created now inside folder "memory"

Hi Shreya! It's great to meet an LLM engineer interested in AI agents and the MCP protocol. If you have any questions or want to dive deeper into any concepts related to AI agents, MCP, or anything else, feel free to ask!

In [5]:
#we create another agent and we give it the same mcp_server with access to memory via params
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, "My name's Shreya. What do you know about me?")
    display(Markdown(result.final_output))

I know that you are Shreya, an LLM engineer. You are learning about AI Agents and the MCP protocol. You understand that MCP is a protocol for connecting agents with tools, resources, and prompt templates, and that it makes integrating AI agents with various capabilities easier. Is there anything else you'd like to add or update about yourself?

### Check the trace:

https://platform.openai.com/traces

![mcp type 1](../images/mcp_all_local_with_local_memory.png)

### The 2nd type of MCP server - runs locally, calls a web service
#### This is the most common way of doing things, by passing an api key to the mcp sever via params as shown in the example below


### Brave Search - apologies - this will need another API key! But it's free again.

https://brave.com/search/api/

Set up your account, and put your key in the .env under `BRAVE_API_KEY`

In [ ]:
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env} # we are passing in our api key this time so we can tell brave about our api key

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server: #even though we are using an online service, we are running it locally on our box using npx
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='brave_web_search', description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, annotations=None),
 Tool(name='brave_local_search', description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations, businesses, restaurants, services, etc. Returns detailed information including:\

In [12]:
instructions = "You are able to search the web for information and briefly summarize the takeaways."
request = f"Please research the latest news on Amazon stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"
model = "gpt-4o-mini"

In [13]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

As of November 6, 2025, Amazon's stock price (AMZN) is at **$243.07**, having decreased by **2.85%** or **$7.13** from the previous day. Here are some key takeaways regarding the outlook for Amazon's stock:

1. **Recent Performance**: After struggling earlier in 2025, Amazon's stock has recently gained some momentum, appreciating **7.35%** over the last five trading days and **4.81%** in the prior week.

2. **Long-term Forecast**: Analysts estimate that Amazon's stock could reach about **$524.67 per share** by the end of 2030, assuming continued revenue growth at a rate of **10% year-over-year**. However, they also note potential margin compression due to increased competition in its AWS division.

3. **Market Dynamics**: The stock's current valuation is below that of some other large-cap retail competitors, raising questions about its attractiveness for long-term investors.

4. **Recent Developments**: Amazon has recently achieved a new high price target amid news regarding partnerships (like with BMW) and developments in its cloud services, which could positively influence future performance.

Overall, while short-term fluctuations are present, the long-term outlook remains optimistic, underscored by growth potential and strategic partnerships.

For more detailed insights, you can refer to the articles linked [here](https://www.fool.com/investing/2025/11/06/is-amazon-stock-a-long-term-buy/) and [here](https://247wallst.com/investing/2025/11/05/amazon-stock-amzn-price-prediction-and-forecast/).

### As usual, check out the trace:

https://platform.openai.com/traces

![mcp_brave_search](../images/mcp_brave_search.png)

## And now the third type: running remotely

It's actually really hard to find a "remote MCP server" aka "hosted MCP server" aka "managed MCP server".

It's not a common model for using or sharing MCP servers, and there isn't a standard way to discover remote MCP servers.

Anthropic lists some remote MCP servers, but these are for paid applications with business users:

https://docs.anthropic.com/en/docs/agents-and-tools/remote-mcp-servers

CloudFlare has tooling for you to create and deploy your own remote MCP servers, but this does not seem to be a common practice:

https://developers.cloudflare.com/agents/guides/remote-mcp-server/


# And back to the 2nd type: the Polygon.io MCP Server

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">PLEASE READ!!-</h2>
            <span style="color:#ff7800;">This service for financial market data has both a FREE plan and a PAID plan, and we can use either depending on your appetite.
            </span>
        </td>
    </tr>
</table>

## NEW SECTION: Introducing polygon.io

Polygon.io is a hugely popular financial data provider. It has a free plan and a paid plan. And it also has an MCP Server!

First, read up on polygon.io on their excellent website, including looking at their pricing:

https://polygon.io

### Polygon.io Part 1: Polygon.io free service (the paid will be totally optional, of course!)

1. Please sign up for polygon.io (top right)  
2. Once signed in, please select "Keys" in the left hand navigation
3. Press the blue "New Key" button
4. Copy the key name
5. Edit your .env file and add the row:

`POLYGON_API_KEY=xxxx`

In [14]:
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")
if not polygon_api_key:
    print("POLYGON_API_KEY is not set")

In [15]:
from polygon import RESTClient
client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

PreviousCloseAgg(ticker='AAPL', close=269.77, high=273.4, low=267.89, open=267.89, timestamp=1762462800000, volume=51203925.0, vwap=270.7137)

### Wrapped into a python module that caches end of day prices

I've made a python module `market.py` that uses this API to look up share prices.

But the free API is quite heavily rate limited - so I've been a bit sneaky; when you ask for a share price, this function retrieves the entire end-of-day equity market, and caches it in our database.


In [ ]:
from market import get_share_price #look at market.py
get_share_price("AAPL")

269.77

In [17]:
# no rate limiting concerns!

for i in range(1000):
    get_share_price("AAPL")
get_share_price("AAPL")

269.77

### And I've made this into an MCP Server

Just as we did with accounts.py; see `market_server.py`

In [18]:
params = {"command": "uv", "args": ["run", "market_server.py"]} #then "get_share_price" has been made into an mcp server in market_server.py
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools

[Tool(name='lookup_share_price', description='This tool provides the current price of the given stock symbol.\n\n    Args:\n        symbol: the symbol of the stock\n    ', inputSchema={'properties': {'symbol': {'title': 'Symbol', 'type': 'string'}}, 'required': ['symbol'], 'title': 'lookup_share_priceArguments', 'type': 'object'}, annotations=None)]

### Let's try it out!

Hopefully gpt-4o-mini is smart enough to know that the symbol for Apple is AAPL

In [19]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple?"
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The current share price of Apple (AAPL) is $269.77.

## Polygon.io Part 2: Paid Plan - Totally Optional!

If you are interested, you can subscribe to the monthly plan to get more up to date market data, and unlimited API calls.

If you do wish to do this, then it also makes sense to use the full MCP server that Polygon.io has released, to take advantage of all their functionality.



In [20]:

""" 
Params of mcp server:
- "uvx" implies python based mcp server
- doesn't need to pip installable, can also be directly from git repo but you need verify that that's the official repo etc to be very sure 
you are using the right thing. make sure you're comfortable with the community traction, the number of stars, the kind of support it has, and so on.
- also pass the api key

Then we ask this mcp server (in this case straight from repo) what tools do you provide
"""
params = {"command": "uvx",
          "args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@v0.1.0", "mcp_polygon"],
          "env": {"POLYGON_API_KEY": polygon_api_key}
          }
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools


[Tool(name='get_aggs', description='\n    List aggregate bars for a ticker over a given date range in custom time window sizes.\n    ', inputSchema={'properties': {'ticker': {'title': 'Ticker', 'type': 'string'}, 'multiplier': {'title': 'Multiplier', 'type': 'integer'}, 'timespan': {'title': 'Timespan', 'type': 'string'}, 'from_': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'format': 'date-time', 'type': 'string'}, {'format': 'date', 'type': 'string'}], 'title': 'From'}, 'to': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'format': 'date-time', 'type': 'string'}, {'format': 'date', 'type': 'string'}], 'title': 'To'}, 'adjusted': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}], 'default': None, 'title': 'Adjusted'}, 'sort': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Sort'}, 'limit': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'Limit'}, 'params': {'anyOf': [{'additionalProperties': True, 'type': 'object'

### Wow that's a lot of tools!

Let's try them out - hopefully the sheer number of tools doesn't overwhelm gpt-4o-mini!

With the $29 monthly plan, we don't have access to some of the APIs, so I've needed to specify which APIs can be called.

If you've splashed out on a bigger plan, feel free to remove my extra constraint..

In [ ]:
"""
We only use the "get_snapshot_ticker" becaus ethat is also available on the free plan. Trying to be careful to simulate paid experience but with tool 
also available on free plan.
"""
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple? Use your get_snapshot_ticker tool to get the latest price."
model = "gpt-4.1-mini" #on "gpt-4o-mini" it did not use the right tool, so we updated to "gpt-4.1-mini" and that made it perform better

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

I am currently unable to access the latest share price for Apple due to data restrictions. Is there anything else related to the stock market that I can help you with?

## Setting up your .env file

If you do decide to have a paid plan, please add this to your .env file to indicate:

`POLYGON_PLAN=paid`

And if you decide to go all the way for the realtime API, then please do:

`POLYGON_PLAN=realtime`

In [22]:
load_dotenv(override=True)

polygon_plan = os.getenv("POLYGON_PLAN")
is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

if is_paid_polygon:
    print("You've chosen to subscribe to the paid Polygon plan, so the code will look at prices on a 15 min delay")
elif is_realtime_polygon:
    print("Wowzer - you've chosen to subscribe to the realtime Polygon plan, so the code will look at realtime prices")
else:
    print("According to your .env file, you've chosen to subscribe to the free Polygon plan, so the code will look at EOD prices")

According to your .env file, you've chosen to subscribe to the free Polygon plan, so the code will look at EOD prices


## And that's it for today!

I've removed the part of this lab that uses the "Financial Datasets" mcp server, because it's inferior - more expensive with fewer APIs.

And this way we get to use the same provider for Free and Paid APIs.

But if you want to see the code, just look in the git history for a prior version.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Explore MCP server marketplaces and integrate your own, using all 3 approaches.
            </span>
        </td>
    </tr>
</table>